# Disease ontologies

lamindb provides access to the following public protein ontologies:

1. [Mondo](https://mondo.monarchinitiative.org/)
2. [Human Disease](https://disease-ontology.org/)

## Setup

In [ ]:
!lamin load test-ontologies

In [ ]:
import lnschema_bionty as lb
import pandas as pd

# adds an entry "human" into an empty instance
lb.settings.organism = "human"

## Bionty objects

Let us create a public knowledge accessor with {meth}`~lnschema_bionty.dev.BioRegistry.bionty`, which chooses a default public knowledge source from {meth}`~lnschema_bionty.BiontySource`. It's a [Bionty](https://lamin.ai/docs/bionty/bionty.bionty) object, which you can think about as a less-capable registry:

In [ ]:
disease_bt = lb.Disease.bionty()
disease_bt

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = disease_bt.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `disease_bt.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = disease_bt.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.alzheimer_disease

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["Alzheimer disease"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = disease_bt.lookup(disease_bt.ontology_id)

In [ ]:
lookup.mondo_0004975

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
disease_bt = lb.Disease.bionty()
disease_bt.search("parkinsons disease").head(3)

By default, search also covers synonyms:

In [ ]:
disease_bt.search("paralysis agitans").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
disease_bt.search("parkinsons disease", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
disease_bt.search(
    "progressive degenerative disorder of the central nervous system", field=disease_bt.definition
).head()

## Standardize disease identifiers

Let us generate a `DataFrame` that stores a number of disease identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
        index=[
            "supraglottis cancer",
            "alexia",
            "trigonitis",
            "paranasal sinus disorder",
            "This disease does not exist",
        ]
    )
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = disease_bt.validate(df_orig.index, disease_bt.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.BiontySource.filter(entity="Disease").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
bionty_source = lb.BiontySource.filter(source="mondo", version="2023-04-04", organism="all").one()
disease_bt = lb.Disease.bionty(bionty_source=bionty_source)
disease_bt

The currently used ontologies can be displayed using:

In [ ]:
lb.BiontySource.filter(currently_used=True).df()